<a href="https://colab.research.google.com/github/AlexandreSarmento/BooleanNetworkBioME/blob/master/C%C3%B3pia_de_TS2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

In [ ]:
#!wget http://www.cs.rice.edu/~nakhleh/TS2B.zip   

In [ ]:
#!unzip TS2B.zip

In [ ]:
!pip install bitarray

In [ ]:
#!pip install booleannet

In [ ]:
!wget http://bonsai.hgc.jp/~mdehoon/software/cluster/Pycluster-1.59.tar.gz

In [ ]:
!tar xzvf Pycluster-1.59.tar.gz

In [ ]:
%cd Pycluster-1.59/

In [ ]:
 !python setup.py install

In [ ]:
!python --version
%cd ../

# iterative KM in depth based on Stenway et al 2015
Here we're just going to binarize. We are adapting a previously developed approach called iterative k-means binarization with a clustering depth (d) of 2 (KM2) or 3 (KM3) for this purpose. This approach was employed because binarized data is able to maintain complex oscillatory behavior in Boolean models constructed from this data, whereas other binarization approaches fail to maintain these features. Once the number of cluster (k) must be greater than the number of experimental measurement (n) k > n. It's well know that k = 2**d,
therefore, here concering to the experimental data provided by Mauro's database we set d = 2, so k = 4.

## download Utils2B

In [ ]:
from google.colab import files
uploaded = files.upload()

## download Breast time serie

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir breast_results
inputFileName = '/content/breast_raw_data'
outputFileName = '/content/breast_results/breast_raw_data'

## download Hepatoma time serie

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir hepatoma_results
inputFileName = '/content/hepatoma_raw_data'
outputFileName = '/content/hepatoma_results/hepatoma_raw_data'

## download Glioma time serie

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir glioma_results
inputFileName = '/content/glioma_raw_data'
outputFileName = '/content/glioma_results/glioma_raw_data'

In [ ]:
inputFileName

## download Proxy time serie

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir breast_results
inputFileName = '/content/breast_raw_data'
outputFileName = '/content/breast_results/breast_raw_data'

In [ ]:
exit()

In [ ]:
# This method of iterative K means binarization was adapted from Berestovsky et al., 2013
# Genes (species should be columns and timepoints should be rows in csv format)
# Include species names as column labels but no timepoints as row labels 
# e.g.:
#       Sp1 Sp2 Sp3
# time1
# time2
# time3


# run by typing 'python KM_Binarization_iterated.py' in ~/NetworkInfFiles-ForManuscript/BinarizationScripts
# This was run with Python 2.7.6 and the following additional packages. It is recommend to use pip to install
# necessary packages: Pycluster, bitarray, 
import math, numpy, sys
import Pycluster
from bitarray import bitarray
from utilsS2B import Binarization, KarnaughMaps, BestFit
import csv, array

originalSeries = {}	# original series stored here as a list
binarySeries = {}	# structure same as the original but with binary series
inputoutput = {}	# keys are combinations, with a list of compbinations that are outputs of it
#fileName = '../Datasets/combinedInterpolatedSamplesNoClinda_natural'
#fileName = sys.argv[1]
number_of_bin = 100
#fileName = '../Datasets/combinedInterpolatedExtrapEquidistantSamplesNoClinda_natural'
#!mkdir glioma_results
#inputFileName = '/content/glioma_nodes_zhang'
#outputFileName = '/content/glioma_results/glioma_zhang_bin_traj'
#def decrementalKMeanBinarization(self, orig, clusters, reduction):
def decrementalKMeanBinarization(orig, clusters, reduction):    
    clusterData = {}
    clusterRes = {}
    binSeries = {}
    if clusters < 2:
            return None
    while clusters >= 2:
            # 'NON' allows you to ignore specific species 
            for k,v in orig.items():
                    if k == "NON":
                            continue
                    data = []
                    for i in range(len(v)):
                            row = [i,v[i]]
                            data.append(row)
                    x = numpy.vstack(v)
                    clusterData[k] = x

                    idx, error, nfound = Pycluster.kcluster(x, nclusters=clusters, method='a', dist='e')
                    clusterRes[k] = (idx, error, nfound)

            # replace the values with the averages of the clusters
            for k,r in clusterRes.items():
                    colors = []
                    idx, error, nfound = r
                    y = clusterData[k]
                    x = numpy.arange(len(y))
                    sizes = {}
                    sums = {}
                    for i in range(clusters):
                            sizes[i] = 0
                            sums[i] = 0
                    for i in range(len(idx)):
                            sizes[idx[i]] += 1
                            sums[idx[i]] += y[i]
                    # culculate sums
                    for i in sums.keys():
                            sums[i] /= sizes[i]
                    # replace all the values with sums in the clusters
                    averaged = []
                    for i in idx:
                            averaged.append(sums[i][0])
                    orig[k] = averaged

            clusters /= 2
    # replace data with 0 and 1
    for k,v in orig.items():
            if k == "NON":
                    continue
            mi = min(v)
            binVals = bitarray()
            for i in v:
                    if i == mi:
                            binVals.append(False)
                    else:
                            binVals.append(True)
            binSeries[k] = binVals
    # reduce binary series
    #print "Iterative K means Binarization"
    #print binSeries
    return binSeries
    #if reduction == 0:
    #        reduced = self.reduceBinarySeries(binSeries)
    #        return reduced
    #if reduction == 1:
    #        reduced = self.reduceBinaryXOR(binSeries)
    #return binSeries

counter = 0
#while (counter < 1000):
while (counter < number_of_bin):
    counter += 1
    print(str(counter) + " of 3 binarizations")
    #print(str(counter) + " of 1000 binarizations")
#def main():
    #binarization method (3 = KM3, 2 = KM2, 1 = KM1)
    bMethod = 2
    #Reduction methods
    reduction = 0
    #input_file = open('genus_abundanceData_transposed.txt', 'r', newline='')
    csvfile = open(inputFileName +'.csv', 'rU') 
    r = csv.reader(csvfile)
    #r = csv.reader(csvfile, dialect='excel-tab')
    #f = open("genus_abundances_crohnsPlusHMP_NoSampleNames.tsv", 'r')
    topLine = r.next()
    order = []
    # grabs each species, 
    for t in topLine:
            # creates an empty list for each species
            originalSeries[t] = []
            order.append(t)
    for line in r:
            if line[0].startswith("#"):
                    continue
            words = line
            # fills the array for each species with the original data points
            for i in range(len(words)):
                    originalSeries[order[i]].append(float(words[i]))
    allConvergence = []
    allTimes = []
    # determines cluster depth (d). Number of clusters, k = 2^d 
    if bMethod > 0:
        clusters = int(math.pow(2,bMethod))
        binarySeries = decrementalKMeanBinarization(dict(originalSeries), clusters, reduction)
    for i in binarySeries:
        sampleSize2= len(binarySeries[i])
    # write binarized data to file
    f = open(outputFileName + str(counter) + '.csv', "wb")
    for item in binarySeries:
        f.write(str(item))
        #print (item + ',')
        #sampleList = list(binarySeries[item])
        values = str(binarySeries[item])
        values  = values.rstrip(')')
        values = values.rsplit('bitarray(')
        values = values[1]
        values = values.lstrip('\'')
        values = values.rstrip('\'')
        valueList = list(values)
        #tmp[1]
        for sample in valueList:
            f.write("," + str(sample))
        #print(str(valueList)+',')
        f.write("\n")

    # Close opened file
    f.close() 
    
#with open('mycsvfile.csv', 'w') as f:  # Just use 'w' mode in 3.x
#    w = csv.DictWriter(f, dict2.keys())
#    for item in dict:
#        w.writer(item)
    #print "sample size: " + str(sampleSize2)
    #print "binarization completed"

# to create zip folder
### aqui você cria uma pasta zip para retirar os arquivos do ambient colab para o usuário do computador

In [ ]:
import shutil

In [ ]:
shutil.make_archive('/content/glioma_results', 'zip', '/content/glioma_results')

In [ ]:
shutil.make_archive('/content/hepatoma_results', 'zip', '/content/hepatoma_results')

In [ ]:
shutil.make_archive('/content/breast_results', 'zip', '/content/breast_results')

In [ ]:
shutil.make_archive('/content/proxy_results_05_1', 'zip', '/content/proxy_results_05_1')

In [ ]:
shutil.rmtree('/content/hepatoma_results', ignore_errors=True)

In [ ]:
# to remove folder such in case
shutil.rmtree('/content/glioma_results', ignore_errors=True)

In [ ]:
shutil.rmtree('/content/breast_results', ignore_errors=True)

In [ ]:
shutil.rmtree('/content/proxy_results_05_1', ignore_errors=True)

# BREAST: KM2-BESTFIT


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=MDAMB231_2.txt bin-method=2 learn-method=BESTFIT maxscore=0.0 solutions=100 

# HepG2: KM2-BESTFIT

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=HEPG2_1.txt bin-method=2 learn-method=BESTFIT maxscore=0.0 solutions=3

# U87: KM2-BESTFIT

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=U87_1.txt bin-method=2 learn-method=BESTFIT maxscore=0.0 solutions=1

# Breast: BASC-BESTFIT

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=MDAMB231_1.txt bin-method=A learn-method=BESTFIT maxscore=0.0 solutions=1

# HepG2:BASC-BESTFIT

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=HEPG2_1.txt bin-method=A learn-method=BESTFIT maxscore=0.0 solutions=1

# U87: BASC-BESTFIT

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python BooleanModeling2post/BinInfer.py input=U87_1.txt bin-method=A learn-method=BESTFIT maxscore=0.0 solutions=1